## Imports

In [1]:
import numpy as np
import pandas as pd
import requests
from collections import OrderedDict

import plotly.figure_factory as ff
from plotly import offline
from plotly.graph_objs import *

pd.options.display.max_columns = 100

# Key is required to access census data
with open("./Census Key.txt", "r") as f:
    key = f.read()
    
year = "2019"

## Functions

In [2]:
# Small proof of concept
fips = ["06021", "06023", "06027",
        "06029", "06033", "06059",
        "06047", "06049", "06051",
        "06055", "06061"]
values = range(len(fips))

fig = ff.create_choropleth(fips=fips, values=values)
fig.layout.template = None

fig.layout.plot_bgcolor = "#fff"
fig.layout.paper_bgcolor = "#fff"
# offline.plot(fig, filename="temp_out.html")
# fig.show()

In [3]:
def json_to_dataframe(response):
    """
        Convert json response to dataframe.
    """
    return pd.DataFrame(response.json()[1:], columns=response.json()[0])

In [4]:
def column_to_int(df, *column_names):
    """
        Convert data type to int.
    """
    for name in column_names:
        df[name] = df[name].astype(int)
        
    return df

In [5]:
def get_endpoints(df, var, num_endpoints=5):
    """ Automatically determine bins for plot legend. """
    endpoints = []
    step = round(1 / (num_endpoints + 2), 3)
    
    quant = step
    for i in range(num_endpoints):
        print(quant)
        endpoints.append(round(df[var].quantile(quant), 3))
        quant += step
        
    return endpoints

In [31]:
def county_map(df, var, endpoints=None):
    """ Generate county map. """
    
    fips = df.index
    values = df[var]
    if endpoints is None:
        endpoints = get_endpoints(df, var)
    
    # Blue
    colorscale = [
        "rgb(220, 220, 220)",
        "rgb(218,218,230)",
        "rgb(185, 185, 220)",
        "rgb(144,148,194)",
        "rgb(101,104,168)",
        "rgb(65, 53, 132)"
    ]
    
    # Red
#     colorscale = [
#         "rgb(220, 220, 220)",
#         "rgb(250, 210, 216)",
#         "rgb(245, 183, 177)",
#         "rgb(236, 112, 99)",
#         "rgb(203, 67, 53)",
#         "rgb(148, 49, 38)"
#     ]

    # Create plot
    fig = ff.create_choropleth(
        fips=fips, values=values,
        binning_endpoints=endpoints, colorscale=colorscale,
        county_outline={"color": "rgb(255,255,255)", "width": 0.5}, round_legend_values=False,
        legend_title=f"{var} ({year})", title=f"{var} by County ({year})",
        show_state_data=True,
        show_hover=True,
        
    )

#     config = dict({"scrollZoom": True})
    fig.layout.template = None

    # Set background
    fig.layout.plot_bgcolor = "#fff"
    fig.layout.paper_bgcolor = "#fff"

    # Output plot (include_plotlyjs required for hover info)
    offline.plot(fig, filename="temp_out.html", include_plotlyjs="https://cdn.plot.ly/plotly-1.42.3.min.js")

## Main

In [7]:
def pull_census_data(var_dict):
    """ Pull down census data for variables provided and return as a DataFrame. """
    var_string = ",".join(var_dict.keys())
    url = f"https://api.census.gov/data/{year}/acs/acs5/?get={var_string}&for=county:*&in=state:*&key={key}"
    response = requests.request("GET", url)

    df = json_to_dataframe(response)
    df = df.rename(columns=var_dict)
    
    return df

In [8]:
# Pull down data from census
base_var_dict = OrderedDict({
    "NAME": "Full Name", 
    "B01001_002E": "Total Male",
    "B01001B_002E": "Black Men",
    "B01001B_012E": "Black Men 45-54",
    "B01001B_013E": "Black Men 55-64",
    "C23002B_010E": "Black Men Over 64", 
    "B01001_001E": "Total Pop", 
    "B02001_003E": "Total Black",
    "B02009_001E": "Partial Black",
    "B19301_001E": "Per capita income",
    "B08014_001E": "Total Vehicle",
    "B08014_002E": "No vehicle",
    "B08014_008E": "Total Vehicle Male",
    "B08014_009E": "Men no vehicle",
    "B08101_001E": "Total Trans",
    "B08101_025E": "Public trans",
    "B28002_002E": "With Internet",
    "B28002_001E": "Total Internet",
    "B28001_001E": "Total Computers",
    "B28001_002E": "Household with Computer",
    "B28001_005E": "Household with Smartphone",
    "B28001_006E": "Household with only Smartphone",
    "B28001_011E": "Household with no Computer",
    "B28009B_001E": "Total Black Household Computer",
    "B28009B_002E": "Black Household with Computer",
    "B28009B_004E": "Black Household with Broadband",
    "B28009B_003E": "Black Household with Dialup",
    "C15002B_002E": "Education Black Male Total",
    "C15002B_003E": "Black Male Less than High School",
    "C15002A_002E": "Education White Male Total",
    "C15002A_003E": "White Male Less than High School",
    "B16010_001E": "Education Total",
    "B16010_002E": "Less than High School",
})

insurance_var_dict = OrderedDict({
    "B27002_018E": "45-54 Male Priv HI Total",
    "B27002_021E": "55-64 Male Priv HI Total",
    "B27002_024E": "65-74 Male Priv HI Total",
    "B27002_027E": "75+ Male Priv HI Total",
    "B27002_019E": "45-54 Male with Priv",
    "B27002_022E": "55-64 Male with Priv",
    "B27002_025E": "65-74 Male with Priv",
    "B27002_028E": "75+ Male with Priv",
    "B27003_019E": "45-54 Male with Pub",
    "B27003_022E": "55-64 Male with Pub",
    "B27003_025E": "65-74 Male with Pub",
    "B27003_028E": "75+ Male with Pub",
    "B27001_002E": "Total Male HI",
    "B27001_018E": "45-54 Male HI Total",
    "B27001_021E": "55-64 Male HI Total",
    "B27001_024E": "65-74 Male HI Total",
    "B27001_027E": "75+ Male HI Total",
    "B27001_020E": "45-54 Male no HI",
    "B27001_023E": "55-64 Male no HI",
    "B27001_026E": "65-74 Male no HI",
    "B27001_029E": "75+ Male no HI",
    "C27001B_001E": "Black HI Total",
    "C27001B_004E": "19- Black no HI",
    "C27001B_007E": "19-64 Black no HI",
    "C27001B_010E": "65+ Black no HI",
    
})

householder_var_dict = OrderedDict({
    "B19001B_001E": "Black Householders",
    "B19001B_002E": "Black Householders less than 10K",
    "B19001B_003E": "Black Householders 10-15K",
    "B19001B_004E": "Black Householders 15-20K",
    "B19001B_005E": "Black Householders 20-25K", 
    "B19001B_006E": "Black Householders 25-30K",
    "B19001B_007E": "Black Householders 30-35K",
    "B19001B_008E": "Black Householders 35-40K",
    "B19037B_037E": "B19037B_037E",
    "B19037B_038E": "B19037B_038E",
    "B19037B_039E": "B19037B_039E",
    "B19037B_040E": "B19037B_040E",
    "B19037B_041E": "B19037B_041E",
    "B19037B_042E": "B19037B_042E",
    "B19037B_043E": "B19037B_043E",
    "B19037B_054E": "B19037B_054E",
    "B19037B_055E": "B19037B_055E",
    "B19037B_056E": "B19037B_056E",
    "B19037B_057E": "B19037B_057E",
    "B19037B_058E": "B19037B_058E",
    "B19037B_059E": "B19037B_059E",
    "B19037B_060E": "B19037B_060E",
})

# All non-terminal df's have the state and county codes removed with iloc
df_list = []
df_list.append(pull_census_data(base_var_dict).iloc[:, :-2])
df_list.append(pull_census_data(insurance_var_dict).iloc[:, :-2])
df_list.append(pull_census_data(householder_var_dict))

df = pd.concat([*df_list], axis=1)

In [9]:
# Get "FIPS" code for state/county 
df["code"] = df["state"] + df["county"]
df = df.set_index("code")

# Split county and state names
temp = df["Full Name"].str.split(",", expand=True)
df["County Name"] = temp[0]
df["State Name"] = temp[1].str.strip()

In [10]:
# Process data
df = column_to_int(df, df.columns[1:-2])

# Convert to int64 to avoid overflow error
df["Total Pop"] = df["Total Pop"].astype(np.int64)
df["Per capita income"] = df["Per capita income"].astype(np.int64)

df["Black Men Over 44"] = df["Black Men 45-54"] + df["Black Men 55-64"] + df["Black Men Over 64"]
df["Percent Black Men Over 44"] = (df["Black Men Over 44"] / df["Total Pop"]) * 100
df["Percent Black"] = (df["Total Black"] / df["Total Pop"]) * 100
df["Percent Partial Black"] = (df["Partial Black"] / df["Total Pop"]) * 100

df["Total income"] = (df["Total Pop"] * df["Per capita income"])

df["45+ Male Priv HI Total"] = df[["45-54 Male Priv HI Total", "55-64 Male Priv HI Total", "65-74 Male Priv HI Total", "75+ Male Priv HI Total"]].sum(axis=1)
df["45+ Male with Priv"] = df[["45-54 Male with Priv", "55-64 Male with Priv", "65-74 Male with Priv", "75+ Male with Priv"]].sum(axis=1)
df["45+ Male with Pub"] = df[["45-54 Male with Pub", "55-64 Male with Pub", "65-74 Male with Pub", "75+ Male with Pub",]].sum(axis=1)

df["45+ Male HI Total"] = df[["45-54 Male HI Total", "55-64 Male HI Total", "65-74 Male HI Total", "75+ Male HI Total"]].sum(axis=1)
df["45+ Male no HI"] = df[["45-54 Male no HI", "55-64 Male no HI", "65-74 Male no HI", "75+ Male no HI"]].sum(axis=1)

df["Percent 45+ Men with Priv"] = df["45+ Male with Priv"] / df["45+ Male Priv HI Total"]
df["Percent 45+ Men with Pub"] = df["45+ Male with Pub"] / df["45+ Male Priv HI Total"]

df["Percent 45+ Men no HI"] = df["45+ Male no HI"] / df["45+ Male HI Total"]

df["Black Householders under 30K"] = (
    df["Black Householders less than 10K"] + 
    df["Black Householders 10-15K"] + 
    df["Black Householders 15-20K"] + 
    df["Black Householders 20-25K"] + 
    df["Black Householders 25-30K"]
)

df["Black Householders under 40K"] = (
    df["Black Householders less than 10K"] + 
    df["Black Householders 10-15K"] + 
    df["Black Householders 15-20K"] + 
    df["Black Householders 20-25K"] + 
    df["Black Householders 25-30K"] + 
    df["Black Householders 30-35K"] + 
    df["Black Householders 35-40K"]
)

df["Black Householders over 44 under 30K"] = (
    df["B19037B_037E"] + 
    df["B19037B_038E"] + 
    df["B19037B_039E"] + 
    df["B19037B_040E"] + 
    df["B19037B_041E"] + 

    df["B19037B_054E"] +
    df["B19037B_055E"] +
    df["B19037B_056E"] +
    df["B19037B_057E"] +
    df["B19037B_058E"]

)

df["Black Householders over 44 under 40K"] = (
    df["B19037B_037E"] + 
    df["B19037B_038E"] + 
    df["B19037B_039E"] + 
    df["B19037B_040E"] + 
    df["B19037B_041E"] + 
    df["B19037B_042E"] + 
    df["B19037B_043E"] + 
    
    df["B19037B_054E"] +
    df["B19037B_055E"] +
    df["B19037B_056E"] +
    df["B19037B_057E"] +
    df["B19037B_058E"] +
    df["B19037B_059E"] +
    df["B19037B_060E"]
)

In [11]:
thresh = 30
black_df = df[df["Percent Black"] > thresh]
other_df = df[df["Percent Black"] <= thresh]

## Population Distribution

In [32]:
county_map(df, "Percent Black", [5, 10, 15, 20, 25])

In [13]:
county_map(df, "Percent Black Men Over 44", [1, 2, 3, 4, 5])

In [14]:
county_map(df, "Total Black", [1000, 5000, 10000, 15000, 20000])

In [15]:
print("Black county count by state")
df[df["Percent Black"] > 30]["State Name"].value_counts()

Black county count by state


Georgia                 67
Mississippi             56
Louisiana               34
North Carolina          32
Virginia                31
Alabama                 27
South Carolina          27
Arkansas                18
Puerto Rico              8
Tennessee                5
Florida                  5
Maryland                 4
New York                 2
Illinois                 2
Missouri                 1
Michigan                 1
Pennsylvania             1
New Jersey               1
District of Columbia     1
Texas                    1
Name: State Name, dtype: int64

In [16]:
interest_states = ["Georgia", "Mississippi", "Louisiana", "North Carolina", "Virginia", "South Carolina", "Alabama", "Arkansas"]
percent_black_in_interest_states = round(df[df["State Name"].isin(interest_states)]["Total Black"].astype(int).sum() / df["Total Black"].sum() * 100, 2)
print(f"The top eight states above contain {percent_black_in_interest_states}% of the total U.S. Black population.")

The top eight states above contain 30.86% of the total U.S. Black population.


## Internet/Computers

In [17]:
black_all_county_with_computer = round(df["Black Household with Computer"].sum() / df["Total Black Household Computer"].sum() * 100, 2)
black_all_county_with_broadband = round(df["Black Household with Broadband"].sum() / df["Total Black Household Computer"].sum() * 100, 2)
black_all_county_with_dialup = round(df["Black Household with Dialup"].sum() / df["Total Black Household Computer"].sum() * 100, 2)
print(f"All county Black households with a computer: {black_all_county_with_computer}%")
print(f"All county Black households with broadband: {black_all_county_with_broadband}%")
print(f"All county Black households with dial-up: {black_all_county_with_dialup}%")

All county Black households with a computer: 90.12%
All county Black households with broadband: 78.54%
All county Black households with dial-up: 0.2%


In [18]:
black_with_computer = round(black_df["Household with Computer"].sum() / black_df["Total Computers"].sum() * 100, 2)
other_with_computer = round(other_df["Household with Computer"].sum() / other_df["Total Computers"].sum() * 100, 2)
print(f"Black county with computer: {black_with_computer}%")
print(f"Other county with computer: {other_with_computer}%")

Black county with computer: 86.81%
Other county with computer: 90.45%


In [19]:
black_with_phone = round(black_df["Household with Smartphone"].sum() / black_df["Total Computers"].sum() * 100, 2)
other_with_phone = round(other_df["Household with Smartphone"].sum() / other_df["Total Computers"].sum() * 100, 2)
print(f"Black county with computer: {black_with_phone}%")
print(f"Other county with computer: {other_with_phone}%")

Black county with computer: 77.66%
Other county with computer: 79.97%


In [20]:
black_with_only_phone = round(black_df["Household with only Smartphone"].sum() / black_df["Total Computers"].sum() * 100, 2)
other_with_only_phone = round(other_df["Household with only Smartphone"].sum() / other_df["Total Computers"].sum() * 100, 2)
print(f"Black county with computer: {black_with_only_phone}%")
print(f"Other county with computer: {other_with_only_phone}%")

Black county with computer: 9.74%
Other county with computer: 6.82%


In [21]:
black_with_internet = round(black_df["With Internet"].sum() / black_df["Total Internet"].sum() * 100, 2)
other_with_internet = round(other_df["With Internet"].sum() / other_df["Total Internet"].sum() * 100, 2)
print(f"Black county with internet: {black_with_internet}%")
print(f"Other county with internet: {other_with_internet}%")

Black county with internet: 77.2%
Other county with internet: 83.41%


## Income

In [22]:
black_per_cap = round(black_df["Total income"].sum() / black_df["Total Pop"].sum())
other_per_cap = round(other_df["Total income"].sum() / other_df["Total Pop"].sum())
print(f"Black county per capita income: {black_per_cap}")
print(f"Other county per capita income: {other_per_cap}")

Black county per capita income: 29843
Other county per capita income: 34307


## Transportation

In [23]:
black_no_vehicle = round(black_df["No vehicle"].sum() / black_df["Total Vehicle"].sum() * 100, 3)
other_no_vehicle = round(other_df["No vehicle"].sum() / other_df["Total Vehicle"].sum() * 100, 3)

print(f"Black county workers with no vehicle access: {black_no_vehicle}%")
print(f"Other county workers with no vehicle access: {other_no_vehicle}%")

Black county workers with no vehicle access: 11.054%
Other county workers with no vehicle access: 3.604%


In [24]:
black_men_no_vehicle = round(black_df["Men no vehicle"].sum() / black_df["Total Vehicle Male"].sum() * 100, 3)
other_men_no_vehicle = round(other_df["Men no vehicle"].sum() / other_df["Total Vehicle Male"].sum() * 100, 3)

print(f"Black county MALE workers with no vehicle access: {black_men_no_vehicle}%")
print(f"Other county MALE workers with no vehicle access: {other_men_no_vehicle}%")

Black county MALE workers with no vehicle access: 10.324%
Other county MALE workers with no vehicle access: 3.554%


In [25]:
# The census website claims the overall figure should be 5% which doesn't match this finding. 
black_public_trans = round(black_df["Public trans"].sum() / black_df["Total Trans"].sum() * 100, 3)
other_public_trans = round(other_df["Public trans"].sum() / other_df["Total Trans"].sum() * 100, 3)

print(f"Black county taking public transit to work: {black_public_trans}%")
print(f"Other county taking public transit to work: {other_public_trans}%")

Black county taking public transit to work: 13.078%
Other county taking public transit to work: 4.19%


## Health Insurance

In [26]:
black_older_men_with_priv = round(black_df["Percent 45+ Men with Priv"].mean() * 100, 2)
other_older_men_with_priv = round(other_df["Percent 45+ Men with Priv"].mean() * 100, 2)
black_older_men_with_pub = round(black_df["Percent 45+ Men with Pub"].mean() * 100, 2)
other_older_men_with_pub = round(other_df["Percent 45+ Men with Pub"].mean() * 100, 2)

print(f"Black county men over 44 with private health insurance: {black_older_men_with_priv}%")
print(f"Other county men over 44 with private health insurance: {other_older_men_with_priv}%")
print(f"Black county men over 44 with public health insurance: {black_older_men_with_pub}%")
print(f"Other county men over 44 with public health insurance: {other_older_men_with_pub}%")

Black county men over 44 with private health insurance: 61.21%
Other county men over 44 with private health insurance: 68.54%
Black county men over 44 with public health insurance: 54.15%
Other county men over 44 with public health insurance: 51.04%


In [27]:
all_county_older_men_no_HI = round(df["Percent 45+ Men no HI"].mean() * 100, 2)
black_older_men_no_HI = round(black_df["Percent 45+ Men no HI"].mean() * 100, 2)
other_older_men_no_HI = round(other_df["Percent 45+ Men no HI"].mean() * 100, 2)

print(f"All county men over 44 with no health insurance: {all_county_older_men_no_HI}%")
print(f"Black county men over 44 with no health insurance: {black_older_men_no_HI}%")
print(f"Other county men over 44 with no health insurance: {other_older_men_no_HI}%")

All county men over 44 with no health insurance: 7.14%
Black county men over 44 with no health insurance: 8.88%
Other county men over 44 with no health insurance: 6.95%


## Householder Income Counts

In [28]:
print(f"Black householders making under 30K: {df['Black Householders under 30K'].sum()}")
print(f"Black householders making under 40K: {df['Black Householders under 40K'].sum()}")
print(f"Black householders older than 44 making under 30K: {df['Black Householders over 44 under 30K'].sum()}")
print(f"Black householders older than 44 making under 40K: {df['Black Householders over 44 under 40K'].sum()}")

Black householders making under 30K: 5649512
Black householders making under 40K: 7265827
Black householders older than 44 making under 30K: 3406973
Black householders older than 44 making under 40K: 4285536


## Education

In [29]:
all_county_less_than_high_school = round(df["Less than High School"].sum() / df["Education Total"].sum() * 100, 2)
all_county_black_male_less_than_high_school = round(df["Black Male Less than High School"].sum() / df["Education Black Male Total"].sum() * 100, 2)
all_county_white_male_less_than_high_school = round(df["White Male Less than High School"].sum() / df["Education White Male Total"].sum() * 100, 2)

print(f"Less than high school degree: {all_county_less_than_high_school}%")
print(f"Black males with less than high school degree: {all_county_black_male_less_than_high_school}%")
print(f"White males with less than high school degree: {all_county_white_male_less_than_high_school}%")

Less than high school degree: 12.12%
Black males with less than high school degree: 15.49%
White males with less than high school degree: 10.9%


In [30]:
black_county_less_than_high_school = round(black_df["Less than High School"].sum() / black_df["Education Total"].sum() * 100, 2)
other_county_less_than_high_school = round(other_df["Less than High School"].sum() / other_df["Education Total"].sum() * 100, 2)
print(f"Black county people with less than high school degree: {black_county_less_than_high_school}%")
print(f"Other county people with less than high school degree: {other_county_less_than_high_school}%")

Black county people with less than high school degree: 14.41%
Other county people with less than high school degree: 11.89%
